# Overview

For HashiCorp Vault, Vault Proxy can be used to proxy API calls. This demo will showcase how Vault Proxy can be used to provide the following capabilities:
1. API Proxy: Functions as a proxy layer to the main Vault cluster.
2. Auth-auth: Vault Proxy authenticates to Vault on behalf of the application.
3. Secret caching: Allows caching of static secrets (KVv1 and KVv2) and dynamic secrets (Dynamic Roles only).
   - Dynamic secrets will be cached based on the TTL expiry. For the demo, we will be using the LDAP engine for dynamic secrets.
   - Static secret caching works with the new Vault event notification system (Enterprise feature) introduced in 1.16. This allows Vault Proxy to refresh the cached entry when write/delete modifications are made to the static secret.

<img src=images/vault-demo-vault-proxy.png>

Ref:
- https://developer.hashicorp.com/vault/docs/agent-and-proxy/autoauth
- https://developer.hashicorp.com/vault/docs/agent-and-proxy/proxy/caching
- https://developer.hashicorp.com/vault/docs/concepts/events

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Check that the various ldap utilities used in this demo is available on your MacBook
# Note bind errors are normal as the OpenLDAP server is not running yet on localhost
ldapadd -V
ldapsearch -V
ldapwhoami -V

# Setup Vault Enterprise and OpenLDAP

To run this notebook in VS Code, chose the Jupyter kernel and then Bash

This assumes also your Vault server is already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables

This also assumes you have the Vault CLI installed.

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest


In [ ]:
# Optional: You can enable file audit device for more information
docker exec -it vault-enterprise /bin/sh -c "mkdir /var/log/vault.d"
docker exec -it vault-enterprise /bin/sh -c "touch /var/log/vault.d/vault_audit.log"
docker exec -it vault-enterprise /bin/sh -c "chown -R vault:vault /var/log/vault.d"
vault audit enable file file_path=/var/log/vault.d/vault_audit.log

# You can run the following command in the container terminal to follow the logs
# tail -f /var/log/vault.d/vault_audit.log
# Or you can run it from outside on your host machine
# docker exec -it vault-enterprise /bin/sh -c "tail -f /var/log/vault.d/vault_audit.log"
# Use Ctrl + C to break

In [ ]:
# Run a OpenLDAP server for the Dynamic Secret caching portion of the demo
# Using 1.4.0 as there is an issue in 1.5.0 where the admin user is missing
# Ref: https://github.com/osixia/docker-openldap/issues/555
export LDAP_PORT=389
export LDAPS_PORT=636
export LDAP_ORG="myorg"
export LDAP_DOMAIN="mydomain.demo"
export LDAP_ADMIN_PASSWORD="Password123"
export LDAP_DN="dc=mydomain,dc=demo"

docker run -d --rm \
  --name openldap \
  -e "LDAP_ORGANISATION=${LDAP_ORG}" \
  -e "LDAP_DOMAIN=${LDAP_DOMAIN}" \
  -e "LDAP_ADMIN_PASSWORD=${LDAP_ADMIN_PASSWORD}" \
  -p ${LDAP_PORT}:${LDAP_PORT} \
  -p ${LDAPS_PORT}:${LDAPS_PORT} \
  osixia/openldap:1.4.0


In [ ]:
# Verify Vault, and OpenLDAP containers are running
docker ps

# Configure Vault for Vault Proxy and Setup Secrets for Demo

## Setup Static Secret

In [ ]:
# Create a KVv2 static secret for the demo, this simulates a credential/secret for an application
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=mypassword

## Setup Auto-Auth using AppRole

In [ ]:
# Enable approle Auth, we will use this for the Vault Proxy's authentication
vault auth enable approle

In [ ]:
# Create a policy that will be tied to the approle used by Vault Proxy
vault policy write kv-proxy-policy - << EOF
path "sys/capabilities-self" {
    capabilities = ["update"]
}
path "secret/data/my-application/*" {
  capabilities = ["create", "update", "read", "list", "subscribe"]
  subscribe_event_types = ["*"]
}

path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
EOF

In [ ]:
# Create the approle for the Vault Proxy with the above policy
# Secret ID usage is currently limited to 10 and token is refreshed every 60 minute
vault write auth/approle/role/kv-proxy-app-role \
    secret_id_ttl=0 \
    token_num_uses=0 \
    token_ttl=60m \
    secret_id_num_uses=10 \
    token_policies=kv-proxy-policy,default
# Show settings on app role
vault read auth/approle/role/kv-proxy-app-role

In [ ]:
# Store roleid value for Vault Agent
vault read -field=role_id auth/approle/role/kv-proxy-app-role/role-id > roleid

In [ ]:
# Store secretid value for Vault Agent
vault write -f -field=secret_id auth/approle/role/kv-proxy-app-role/secret-id > secretid

## Configure and start Vault Proxy

In [ ]:
# Create the Vault Proxy configuration file. The Vault container is exposed via local port 8200.
# For the demo, we will be running Vault Proxy locally in a terminal on port 8100.
tee proxy.hcl <<EOF
pid_file = "./pidfile"

vault {
  address = "http://127.0.0.1:8200"
  retry {
    num_retries = 5
  }
}

auto_auth {
  method {
    type = "approle"
    config = {
      role_id_file_path = "roleid"
      secret_id_file_path = "secretid"
      remove_secret_id_file_after_reading = false
    }
  }
}
cache {
  cache_static_secrets = true
  disable_caching_dynamic_secrets = false
}

api_proxy {
  use_auto_auth_token = "force"
}

listener "tcp" {
    address = "127.0.0.1:8100"
    tls_disable = true
}
log_level="TRACE"
EOF

In [ ]:
# Run the vault proxy in a new terminal window.
# Type ^ + Shift + ` to open a new terminal windows in VS Code
# Copy and paste the follow command to start the Vault proxy
# vault proxy -config=proxy.hcl

# Testing Vault Proxy Auto-Auth and Static Secret Caching

When doing the API call below, you will notice that there is no Vault token required.  This shows the auto-auth feature in action.

In [ ]:
# You can open the docker dashboard and view the Vault container logs side by side as you run this
# Run an API call using curl to simulate the application accessing the secret
# You can see from the terminal window that the Vault Proxy will retrieve the secret from Vault on the first call.
# On subsequent calls, it retreives the secret from the Vault Proxy cache and there is no calls made to Vault subsequently.
#curl --header "X-Vault-Token: $VAULT_TOKEN" http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials
curl http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials | jq

In [ ]:
# Configure CLI to point to Vault Proxy
export VAULT_PORT=8100
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
unset VAULT_TOKEN

In [ ]:
# Test retrieving KV secret via Vault CLI read command.  
# Verify that the credential is returned from the cache.
vault read secret/data/my-application/my-credentials

## Vault Proxy with Vault event notification system

We will now see the how Vault Proxy is notified of changes on the secret via the Vault event notification system. This is used to refresh the cached entry.

In [ ]:
# Configure CLI to point to Vault Server
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

In [ ]:
# Update the password. You can run either of the following commands to alternate updates.
# Note in the terminal that the Vault Proxy will be notified of the write event and it will refresh the cache

In [ ]:
# Update password to '12345678'
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=12345678

In [ ]:
# Verify that the new password '12345678' is returned from the cache immediately.
curl http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials | jq

In [ ]:
# Update password to 'abcdefgh'
vault kv put -mount=secret my-application/my-credentials userid=johndoe password=abcdefgh

In [ ]:
# Verify that the new password abcdefgh is returned from the cache immediately.
curl http://127.0.0.1:8100/v1/secret/data/my-application/my-credentials | jq

# Testing Vault Proxy with Dynamic Secret Caching

For demo purposes, we will be using the LDAP engine with an OpenLDAP server to demonstrate how Vault Proxy works with dynamic engines.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/ldap
- https://developer.hashicorp.com/vault/tutorials/secrets-management/openldap

## Populate OpenLDAP with Test Objects

In [ ]:
# Populate LDAP server with test objects
export LDAP_TEST_USER="john"
export LDAP_TEST_USER_PASSWORD="Password123"
export LDAP_TEST_GROUP="finance"

tee openldapsetup.ldif <<EOF
dn: ou=groups,$LDAP_DN
objectClass: organizationalunit
objectClass: top
ou: groups
description: groups of users

dn: ou=users,$LDAP_DN
objectClass: organizationalunit
objectClass: top
ou: users
description: users

dn: cn=$LDAP_TEST_GROUP,ou=groups,$LDAP_DN
objectClass: groupofnames
objectClass: top
description: testing group for dev
cn: $LDAP_TEST_GROUP
member: cn=$LDAP_TEST_USER,ou=users,$LDAP_DN

dn: cn=$LDAP_TEST_USER,ou=users,$LDAP_DN
objectClass: person
objectClass: top
cn: $LDAP_TEST_USER
sn: $LDAP_TEST_USER
memberOf: cn=$LDAP_TEST_GROUP,ou=groups,$LDAP_DN
userPassword: $LDAP_TEST_USER_PASSWORD
EOF

ldapadd -c -w $LDAP_ADMIN_PASSWORD -D "cn=admin,$LDAP_DN" -f openldapsetup.ldif

In [ ]:
# Show the configured LDAP objects
echo "LDAP DN: $LDAP_DN"
echo "LDAP Admin Password: $LDAP_ADMIN_PASSWORD"
ldapsearch -x -b $LDAP_DN -D "cn=admin,$LDAP_DN" -w $LDAP_ADMIN_PASSWORD

## Configure Vault's LDAP Engine

In [ ]:
# Enable the LDAP secrets engine
vault secrets enable ldap

In [ ]:
# As both Vault and OpenLDAP is running on docker, Vault will be connecting to OpenLDAP via the docker bridge network
# Obtain IP address of the OpenLDAP server for configuration
export OPENLDAP_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' openldap)
echo "OpenLDAP Server IP Address is: $OPENLDAP_IP"

export OPENLDAP_URL=${OPENLDAP_IP}:${LDAP_PORT}
echo "OpenLDAP URL: $OPENLDAP_URL"
echo "LDAP DN: $LDAP_DN"
echo "LDAP Admin Password: $LDAP_ADMIN_PASSWORD"

vault write ldap/config \
    binddn="cn=admin,$LDAP_DN" \
    bindpass=$LDAP_ADMIN_PASSWORD \
    schema="openldap" \
    url=ldap://$OPENLDAP_URL \
    skip_static_role_import_rotation=false
echo
vault read ldap/config



In [ ]:
# Rotate the LDAP root credential (Optional)
vault write -f ldap/rotate-root

In [ ]:
# Note that even though we have rotated the root password, there is an existing behaviour in 
# the OpenLDAP container that allows us to login using the old password
# https://github.com/osixia/docker-openldap/issues/161
# We will be using this behavor to show the rotated password. This would not be possible in normal circumstances.
echo "LDAP DN: $LDAP_DN"
export LDAP_ADMIN_PASSWORD="Password123"
# Store the new password
export LDAP_ADMIN_PASSWORD=$(ldapsearch -LLL -o ldif-wrap=no -x -b "cn=admin,$LDAP_DN" -D "cn=admin,$LDAP_DN" -w $LDAP_ADMIN_PASSWORD | grep userPassword | awk '{print $2}' | base64 -d)
echo "New LDAP Admin Password: $LDAP_ADMIN_PASSWORD"


In [ ]:
# Show that the admin is able to login with the new rotated password
echo "LDAP DN: $LDAP_DN"
echo "LDAP Admin Password: $LDAP_ADMIN_PASSWORD"
ldapsearch -LLL -x -b "cn=admin,$LDAP_DN" -D "cn=admin,$LDAP_DN" -w $LDAP_ADMIN_PASSWORD

## Configure a LDAP Static Role

This represents a fixed LDAP user where Vault manages the password rotation.

In [ ]:
# Configure static role for the LDAP user created earlier
export LDAP_STATIC_ROLE="my-static-role"
export LDAP_STATIC_ROLE_TTL="1h"

echo "LDAP_DN: $LDAP_DN"
echo "LDAP_TEST_USER: $LDAP_TEST_USER"
vault write ldap/static-role/$LDAP_STATIC_ROLE \
    dn="cn=$LDAP_TEST_USER,ou=users,$LDAP_DN" \
    username="$LDAP_TEST_USER" \
    rotation_period="$LDAP_STATIC_ROLE_TTL"
echo
vault read ldap/static-role/my-static-role

In [ ]:
# Show the rotation of the static role password
vault write -f ldap/rotate-role/$LDAP_STATIC_ROLE

In [ ]:
# Read my-static-role password
vault read ldap/static-cred/$LDAP_STATIC_ROLE
export LDAP_TEST_USER_PASSWORD=$(vault read -field=password ldap/static-cred/$LDAP_STATIC_ROLE)
echo
echo "LDAP_TEST_USER_PASSWORD: $LDAP_TEST_USER_PASSWORD"

In [ ]:
# Show that the test user is able to login with the new rotated password
echo "LDAP DN: $LDAP_DN"
echo "LDAP_TEST_USER: $LDAP_TEST_USER"
#export LDAP_TEST_USER_PASSWORD="Password123"
echo "LDAP_TEST_USER_PASSWORD: $LDAP_TEST_USER_PASSWORD"
echo
ldapwhoami -vvv -h localhost -p 389 -D "cn=$LDAP_TEST_USER,ou=users,$LDAP_DN" -x -w "$LDAP_TEST_USER_PASSWORD"


In [ ]:
# Update the Vault Proxy policy to allow reading of the LDAP both static (ldap/static-cred/*) and dynamic (ldap/cred/*) role credentials
vault policy write kv-proxy-policy - << EOF
path "sys/capabilities-self" {
    capabilities = ["update"]
}
path "secret/data/my-application/*" {
  capabilities = ["create", "update", "read", "list", "subscribe"]
  subscribe_event_types = ["*"]
}

path "ldap/static-cred/*" {
  capabilities = ["read"]
}

path "ldap/creds/*" {
  capabilities = ["read"]
}

path "sys/events/subscribe/*" {
  capabilities = ["read"]
}
EOF

In [ ]:
# You will notice that this call is a pass through.
# Note: Vault Proxy currently does not cache the static role values.
#       You will see a "forwarding request to Vault" message in the terminal.
curl http://127.0.0.1:8100/v1/ldap/static-cred/$LDAP_STATIC_ROLE | jq

## Configure a LDAP Dynamic Role

For dynamic roles, Vault creates a new LDAP account with a specified TTL. Upon expiry of the TTL, Vault will remove the created LDAP account.

In [ ]:
# Set the LDAP dynamic role name and TTL
export LDAP_DYNAMIC_ROLE="my-dynamic-role"
export LDAP_DYNAMIC_ROLE_TTL="1h"

# Generate the LDIF files for the dynamic role
# cn - common name. sn - surname.
# Will be creating the user under the test group
tee creation.ldif <<EOF
dn: cn={{.Username}},ou=users,$LDAP_DN
objectClass: person
objectClass: top
cn: $LDAP_DYNAMIC_ROLE
sn: $LDAP_DYNAMIC_ROLE
memberOf: cn=$LDAP_TEST_GROUP,ou=groups,$LDAP_DN
userPassword: {{.Password}}
EOF

tee deletion.ldif <<EOF
dn: cn={{.Username}},ou=users,$LDAP_DN
changetype: delete
EOF

# Create the LDAP dynamic role in Vault
vault write ldap/role/$LDAP_DYNAMIC_ROLE \
    creation_ldif=@creation.ldif \
    deletion_ldif=@deletion.ldif \
    rollback_ldif=@deletion.ldif \
    default_ttl=$LDAP_DYNAMIC_ROLE_TTL

rm creation.ldif deletion.ldif

In [ ]:
# Request a dynamic secret. You will see a message "storing dyamic secret response in cache" message in the terminal.
curl -o dynamic-creds.json http://127.0.0.1:8100/v1/ldap/creds/$LDAP_DYNAMIC_ROLE 

# Verify that the dynamic user credentials are valid
export LDAP_DYNAMIC_USER=$(jq -r .data.username < dynamic-creds.json)
export LDAP_DYNAMIC_PASSWORD=$(jq -r .data.password < dynamic-creds.json)
echo "LDAP_DYNAMIC_USER: $LDAP_DYNAMIC_USER"
echo "LDAP_DYNAMIC_PASSWORD: $LDAP_DYNAMIC_PASSWORD"
echo
ldapwhoami -vvv -h localhost -p 389 -D "cn=$LDAP_DYNAMIC_USER,ou=users,$LDAP_DN" -x -w "$LDAP_DYNAMIC_PASSWORD"

In [ ]:
# Stop Vault to simulate an outage
docker stop vault-enterprise

In [ ]:
# Request a dynamic secret again. Notice that you are still able to get the cached credentials.
# You will see a message "returning cached dynamic secret response" message in the terminal.
curl http://127.0.0.1:8100/v1/ldap/creds/$LDAP_DYNAMIC_ROLE | jq

# Cleanup

In [ ]:
# Cleanup
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Remove temp files
rm roleid
rm secretid
rm proxy.hcl
rm pidfile
rm openldapsetup.ldif
rm dynamic-creds.json

# Stop Vault and OpenLDAP
docker stop openldap
docker stop vault-enterprise

In [ ]:
# Terminate the Vault Proxy in the terminal window by using Ctrl + C

# Other Useful Commands

In [ ]:
# To subscribe to KVv2 write events
vault events subscribe kv-v2/data-write

In [ ]:
# Retrieve KV secret via Vault CLI kv command.
# Note this command does a mount check back to the Vault server and will fail if the Vault Server is not contactable by the Vault Proxy.
vault kv get -mount=secret my-application/my-credentials

In [ ]:
# Kill any background Vault Proxy process
pgrep -f vault | xargs kill